In [1]:
from pymongo import MongoClient
import nltk
from nltk.corpus import stopwords
import enchant
from string import digits
from collections import Counter
import json

In [2]:
client = MongoClient()

In [3]:
db = client.precog_fb_data

In [4]:
events = db.events

In [5]:
categories = ['travel', 'food', 'music', 'art', 'education']

In [6]:
all_events = {}
for cat in categories:
    big_string = ''
    temp_cursor = events.find({'category': cat})
    for item in temp_cursor:
        try:
            temp = item['description']
        except:
            pass
        big_string += temp
    all_events[cat] = big_string.replace("\n", " ").lower()

In [7]:
print len(all_events['art']), "\n", all_events['art'][:25]

317467 
art weekend at the common


In [8]:
d = enchant.Dict("en_US")
word_list = {}
stop = set(stopwords.words('english'))

In [9]:
all_events['art']

u'art weekend at the commons! come and experience a brand new creative event in our neighborhood. \U0001f3a8  let\u2019s support our local talents!  ateliers --------- come say hello to these artists and take a sneak peek at how they work and bring their pieces to life!  o terawat alex face nakrob/moon/mars/nut. supmanee c. pianissimo press   lovesyrup him hp nut.dao tuna dunn ploypuyik & yoky aeicha papeterie vanont ruk & kimmy  unblanking the canvas  --------------------------  12 artists + 1 canvas = live masterpiece! each artist will take turns applying their craft onto the same canvas - don\u2019t miss the final reveal (on both days)!  workshops ------------- join these creative sessions! *advance booking  > gatherings@thecommonsbkk.com  sat, sep 9th (12-1pm, 1:30-2:30pm) - floral astrological signs workshop  sat, sep 9th (12-1pm, 1:30-2:30pm) -  go with the flow - resin workshop  sat, sep 9th (3:30-6pm) - colors & layers paper collage workshop *walk-ins welcome*  sun, sep 10th (2

In [10]:
for cat in categories:
    english_string = ''
    english_list = []
    temp = all_events[cat].replace(".",'').replace("-",'')
    for word in temp.split():
        if d.check(word):
            english_string += word + " "
    english_string = english_string.encode("utf-8")
    english_string = english_string.translate(None, digits)
    english_list = [ix for ix in english_string.split() if ix not in stop]
    english_list = [ix for ix in english_list if len(ix)>2 ]
    print len(english_list), "\n", english_list[:25]
    word_list[cat] = english_list 

10082 
['come', 'airport', 'discover', 'fantastic', 'destinations', 'travel', 'speak', 'vast', 'range', 'tour', 'operator', 'let', 'inspire', 'book', 'dream', 'holiday', "you've", 'always', 'wanted', 'team', 'airport', 'travel', 'offering', 'exclusive', 'offers']
12157 
['kids', 'corn', 'live', 'family', 'friendly', 'alcohol', 'food', 'trucks', 'free', 'park', 'going', 'peter', 'park', 'food', 'second', 'free', 'artisan', 'food', 'market', 'locally', 'purveyed', 'goods', 'food', 'trucks', 'vegan']
9396 
['praise', 'concert', 'god', 'gate', 'details', 'watch', 'space', 'details', 'sign', 'form', 'receive', 'notifications', 'heading', 'ultra', 'music', 'festival', 'deadliest', 'combination', 'piedmont', 'park', 'get', 'latest', 'information', 'contests', "who's"]
13226 
['art', 'weekend', 'come', 'experience', 'brand', 'new', 'creative', 'event', 'neighborhood', 'support', 'local', 'ateliers', 'come', 'say', 'hello', 'artists', 'take', 'sneak', 'peek', 'work', 'bring', 'pieces', 'face', 

In [24]:
word_frequency = {}
for cat in categories:
    c = Counter(word_list[cat]).most_common(200)
    temp = [{'word': x[0], 'amt': x[1]} for x in c]
    word_frequency[cat] = temp


In [18]:
# OLD CODE
# word_frequency = {}
# for cat in categories:
#     temp = word_list[cat]
#     c = Counter(temp)
#     temp_list =[]
#     for item in c:
#         temp_list.append({'amt': c[item], 'word': item})
#     word_frequency[cat] = temp_list

In [25]:
with open('word_freq.json', 'w') as fp:
    json.dump(word_frequency, fp)

In [30]:
piechart_data_dic = {}
metrics = ['attending_count', 'interested_count', 'noreply_count']
for ix in metrics:
    temp_dic = {}
    for cat in categories:
        cursor = events.find({'category': cat})
        total = 0
        for item in cursor:
            total += item[ix]
        temp_dic[cat] = total
    piechart_data_dic[ix] = temp_dic

In [33]:
temp_dic = {}
for cat in categories:
    cursor = events.find({'category': cat})
    total = 0
    for item in cursor:
        total += item['attending_count']
    temp_dic[cat] = total_attendes/cursor.count()
piechart_data_dic['average'] = temp_dic

In [34]:
piechart_data_dic

{'attending_count': {'art': 370795,
  'education': 45314,
  'food': 309746,
  'music': 604307,
  'travel': 53456},
 'average': {'art': 1376,
  'education': 1811,
  'food': 1401,
  'music': 1395,
  'travel': 1518},
 'interested_count': {'art': 1488531,
  'education': 231897,
  'food': 2001686,
  'music': 2166801,
  'travel': 355444},
 'noreply_count': {'art': 385915,
  'education': 51958,
  'food': 310405,
  'music': 746477,
  'travel': 103668}}

In [35]:
with open('donut_data.json', 'w') as fp:
    json.dump(piechart_data_dic, fp)

In [47]:
geo_data = {}
for cat in categories:
    cat_list = [['Lat', 'Long', 'Event']]
    cursor = events.find({'category': cat})
    for event in cursor:
        event_list = []
        try:
            event_list.append(event['place']['location']['latitude'])
            event_list.append(event['place']['location']['longitude'])
            event_list.append(event['name'])
            cat_list.append(event_list)
        except:
            pass
    geo_data[cat] = cat_list

In [50]:
with open('geo_data.json', 'w') as fp:
    json.dump(geo_data, fp)

In [55]:
cursor = events.find({})

In [57]:
cursor[0]

{u'_id': ObjectId('59b14b04117e07444fe1051f'),
 u'attending_count': 644,
 u'can_guests_invite': True,
 u'can_viewer_post': True,
 u'category': u'food',
 u'cover': {u'id': u'1682607055142917',
  u'offset_x': 0,
  u'offset_y': 50,
  u'source': u'https://scontent.xx.fbcdn.net/v/t1.0-9/20228332_1682607055142917_1000590449691117850_n.jpg?oh=ce2a843b293ed87a2c1b08d9b2c25ec4&oe=5A4D5150'},
 u'declined_count': 0,
 u'description': u'Kids area, corn hole, live music, family friendly games, DJ, alcohol samples, food trucks- and MORE!\nFree admisson, only $3 to park (with 100% going to the CTFD foundation)',
 u'end_time': u'2017-09-07T21:00:00-0400',
 u'guest_list_enabled': True,
 u'id': u'505479789796109',
 u'interested_count': 5582,
 u'is_canceled': False,
 u'is_draft': False,
 u'is_page_owned': True,
 u'is_viewer_admin': False,
 u'maybe_count': 5582,
 u'name': u'Food Truck Rally',
 u'noreply_count': 531,
 u'owner': {u'id': u'772268349510130',
  u'name': u'Sterling Heights Regional Chamber of Co

In [59]:
for ix in range(10):
    print cursor[ix]['start_time'], "### ",cursor[ix]['timezone']

2017-09-07T16:00:00-0400 ###  America/Detroit
2017-10-13T17:00:00-0400 ###  America/New_York
2017-09-02T12:00:00+0900 ###  Asia/Ulaanbaatar
2017-09-07T10:00:00+0300 ###  Europe/Bucharest
2017-09-08T16:00:00-0400 ###  America/Toronto
2017-09-09T11:00:00-0400 ###  America/New_York
2017-07-23T17:00:00-0400 ###  America/New_York
2017-09-14T16:00:00-0400 ###  America/New_York
2017-09-23T18:30:00-0700 ###  America/Los_Angeles
2018-04-01T16:00:00-0400 ###  America/New_York
